In [1]:
import requests 
import re 
from bs4 import BeautifulSoup 
import pandas as pd

In [2]:
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}
links = []
page_num = 1
while True :
    page_two = requests.get(f'https://www.redfin.com/city/30749/NY/New-York/page-{page_num}' , headers = headers)
    content = page_two.content
    soup = BeautifulSoup(content, features='html.parser')
    home_info = soup.find_all('div' , class_ = "HomeCardContainer defaultSplitMapListView")
    for i in range(len(home_info)):
        links.append('https://www.redfin.com' + home_info[i].find('a').attrs['href'])
        
    print(f'page {page_num} : Done')
    page_num += 1
    if page_num == 10:
        print('done all')
        break

page 1 : Done
page 2 : Done
page 3 : Done
page 4 : Done
page 5 : Done
page 6 : Done
page 7 : Done
page 8 : Done
page 9 : Done
done all


In [4]:
# Declare list for each feature
HOA_Dues = []
Style = []
pro_type = []
Comm = []
price = []
Bads = []
Baths = []
sqft = []
address = []
year_built = []

In [5]:
def extract_info(feature , list_name):
    """function to extract information of each type"""
    if feature ==  [] or feature == ():
        list_name.append('None')
    else:
        list_name.append(feature[0])

In [17]:
for link in links :
    page = requests.get(link , headers = headers)
    content = page.content
    soup = BeautifulSoup(content, features='html.parser')
    
    main_content = soup.find('div' , class_ = 'keyDetailsList').text
    main_info = soup.find('div' , class_ = 'home-main-stats-variant')
    address_info = soup.find('h1' , class_ = 'full-address')
    
    #get address of houses 
    address.append(address_info.contents[0].text)
    
    #get address of houses
    price.append(main_info.contents[0].contents[0].text)
    
    #get number of bads of houses
    Bads.append(main_info.contents[1].text)
    
    #get number of baths of houses
    Baths.append(main_info.contents[2].text)
    
    #get square feet of houses
    sqft.append(main_info.contents[3].text)
    
    #get homeowners association of houses
    HOA = re.findall('([0-9]\d+/month|[0-9],\d+/month)' , main_content)
    extract_info(HOA , HOA_Dues)
    
    #get property type of houses
    typer = re.findall('Property Type(.*?)(?=HOA|Year|Baths)' , main_content)
    extract_info(typer , pro_type)
    
    #get style of houses
    style = re.findall('Style(.*?)Community' , main_content)
    extract_info(style , Style)
    
    #get community of houses
    Community = re.findall('Community(.*?)Lot' , main_content)
    extract_info(Community , Comm)
    
    #get year built of houses
    year = re.findall('Year Built(\d+)' , main_content)
    extract_info(year , year_built)

In [82]:
#Make dataframe with values of each feature
data = pd.DataFrame(
    {'Address' : address , 'Beds' : Bads , 'Baths' : Baths , 'Sqft' : sqft , 'HOA(Month)': HOA_Dues,'Property Type': pro_type,
     'Style': Style , 'Community' : Comm , 'YearBuilt': year_built , 'Price' : price
    })

data

,Address,Beds,Baths,Sqft,HOA(Month),Property Type,Style,Community,YearBuilt,Price
0,"221 Beach 80th St Unit 5H,",2Beds,1Bath,885Sq Ft,784/month,Condo,Mid-Rise,None,2005,"$359,000"
1,"251 Seaman Ave Unit 5F,",2Beds,2Baths,—Sq Ft,"1,297/month",Condo (Co-op),Prewar,Inwood,1938,"$649,000"
2,"416 W 52nd St #520,",1Bed,1Bath,820Sq Ft,756/month,Condo,Prewar,Hells Kitchen,1940,"$1,180,000"
3,"225 Fifth Ave Ph -H,",3Beds,3Baths,"1,987Sq Ft","2,740/month",Condo,Prewar,NoMad,1907,"$3,495,000"
4,"12 Agate Ct,",4Beds,3.5Baths,"2,720Sq Ft",None,Townhouse,Prewar,Stuyvesant Heights,1899,"$1,700,000"
...,...,...,...,...,...,...,...,...,...,...
346,"441 79th St,",4Beds,2Baths,"1,326Sq Ft",None,Single Family,Duplex,Brooklyn,1925,"$1,458,000"
347,"144-77 Barclay Ave Unit 7C,",2Beds,2Baths,"1,220Sq Ft",412/month,Condo,Mid-Rise,-,2007,"$730,000"
348,"67 Loring Ave,",3Beds,2.5Baths,"1,600Sq Ft",None,"Colonial, A",None,Annandale,1978,"$739,900"
349,"58-36 150th St,",3Beds,1.5Baths,"1,260Sq Ft",None,Townhouse,Townhouse,Flushing,1950,"$999,888"


In [83]:
#Remove (Beds-Bad- —) from Beds column
data['Beds'] = data['Beds'].str.replace('Beds' ,'')
data['Beds'] = data['Beds'].str.replace('Bed' , '')
data['Beds'].replace('—' , 'None' , inplace = True)

#Remove (Beths-Bath) from Baths column
data['Baths'] = data['Baths'].str.replace('Baths' ,'')
data['Baths'] = data['Baths'].str.replace('Bath' , '')

#Remove (Sq Ft- —Sq Ft- —) from Square feet column
data['Sqft'] = data['Sqft'].str.replace('Sq Ft' ,'')
data['Sqft'] = data['Sqft'].str.replace('——Sq Ft' , 'None')
data['Sqft'].replace('—' , 'None' , inplace = True)

#Remove /month from HOA(Month) column
data['HOA(Month)'] = data['HOA(Month)'].str.replace('/month' ,'')

#Replace - with None in community column
data['Community'].replace('-' , 'None' , inplace = True)

#Remove $ from price column
data['Price'] = data['Price'].str.replace('$' ,'')

C:\Users\Tawagny\AppData\Local\Temp/ipykernel_13092/1828280642.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Price'] = data['Price'].str.replace('$' ,'')


In [84]:
data[9:20]

,Address,Beds,Baths,Sqft,HOA(Month),Property Type,Style,Community,YearBuilt,Price
9,"55 Wall St #713,",None,1,658,"1,230",Condo,Prewar,Financial District,1842,"675,000"
10,"3 Riverside Dr,",9,11.5,"19,251",None,Townhouse,Prewar,Upper West Side,1899,"12,995,000"
11,"310 Lexington Ave Unit 10-H,",None,1,None,"1,384",Condo (Co-op),None,Murray Hill,1956,"525,000"
12,"1135 Rogers Ave #1,",7,4,"2,760",None,Townhouse,None,Flatbush,1899,"998,000"
13,"555 W End Ave Unit 4-W,",4,4.5,"2,716","3,481",Condo,Prewar,Upper West Side,1908,"7,995,000"
14,"497 3rd St Ph,",3,3,"1,890","2,021",Condo,Prewar,Park Slope,1921,"3,099,000"
15,"425 W 50th St Unit 14-D,",1,2,"1,416","2,462",Condo,Prewar,Hells Kitchen,1927,"1,875,000"
16,"575 Park Ave #1401,",2,2,None,"6,735",Condo (Co-op),Prewar,Lenox Hill,1927,"999,000"
17,"150 W 26th St #202,",2,2.5,"1,800",859,Condo,Prewar,Chelsea,1923,"2,995,000"
18,"530 E 76th St Unit 34/35AB,",4,3.5,"2,930","5,433",Condo,None,Upper East Side,1986,"3,000,000"
